<a href="https://colab.research.google.com/github/seunghyun3309/this_is_data_analysis_practice/blob/master/KW_MMDS_Colab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# KW_MMDS - Colab 2
## Frequent Pattern Mining in Spark

### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     |████████████████████████████████| 204.2MB 80kB/s 
     |████████████████████████████████| 204kB 46.6MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=2d8e671c1390763e11f479d3f337a468f753fc4579301ac8c73c619d9078f3d6
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark
The following additional packages will be installed:
  openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra
  fonts-ipafont-gothic fonts-ipafont-mincho fonts-wqy-microhei
  fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 2 newly installed, 0 to remove and 6 not upgraded.
Need to get 35.8 MB of archives.
After this operation, 140 MB of additional disk space will be used.
Selecting previously unselected package openjdk-8-jre-headless:amd64.
(Reading 

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [6]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [7]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [8]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Use the Spark Dataframe API to join 'products' and 'orders', so that you will be able to see the product names in each transaction (and not only their ids).  Then, group by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.
Use the Python example in the documentation, and train a model with 

```minSupport=0.01``` and ```minConfidence=0.5```



1. Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [9]:
# YOUR CODE HERE
# YOUR CODE HERE
orders_joined = orders.join(products, products.product_id == orders.product_id)
orders_joined = orders_joined.selectExpr(["order_id", "product_name"])
orders_joined.show(truncate=False)

# grouping
df = orders_joined.groupBy('order_id').agg(collect_set('product_name').alias('items'))
df.show(truncate=False)

from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.01, minConfidence=0.5)
model = fpGrowth.fit(df)

print("Frequent Itemsets : ", model.freqItemsets.count())
print("Association Rules : ", model.associationRules.count())

+--------+---------------------------------------------+
|order_id|product_name                                 |
+--------+---------------------------------------------+
|1       |Bulgarian Yogurt                             |
|1       |Organic 4% Milk Fat Whole Milk Cottage Cheese|
|1       |Organic Celery Hearts                        |
|1       |Cucumber Kirby                               |
|1       |Lightly Smoked Sardines in Olive Oil         |
|1       |Bag of Organic Bananas                       |
|1       |Organic Hass Avocado                         |
|1       |Organic Whole String Cheese                  |
|36      |Grated Pecorino Romano Cheese                |
|36      |Spring Water                                 |
|36      |Organic Half & Half                          |
|36      |Super Greens Salad                           |
|36      |Cage Free Extra Large Grade AA Eggs          |
|36      |Prosciutto, Americano                        |
|36      |Organic Garnet Sweet 

2. What is the most frequent item?

In [22]:
model.freqItemsets.sort("freq",ascending=False).show(1,truncate=False)

+------------------------+-----+
|items                   |freq |
+------------------------+-----+
|[Banana]                |18726|
|[Bag of Organic Bananas]|15480|
|[Organic Strawberries]  |10894|
|[Organic Baby Spinach]  |9784 |
|[Large Lemon]           |8135 |
+------------------------+-----+
only showing top 5 rows



3. Now retrain the FP-growth model changing only 
```minsupport=0.001``` 
and compute how many frequent itemsets and association rules were generated.


In [23]:
# YOUR CODE HERE
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="items", minSupport=0.001, minConfidence=0.5)
model = fpGrowth.fit(df)
print("Frequent Itemsets : ", model.freqItemsets.count())
print("Association Rules : ", model.associationRules.count())

Frequent Itemsets :  4444
Association Rules :  11


4. Print all the association rules of the above problem.

In [24]:
# YOUR CODE HERE
model.associationRules.show(truncate=False)

+-----------------------------------------------------------------+------------------------+------------------+------------------+
|antecedent                                                       |consequent              |confidence        |lift              |
+-----------------------------------------------------------------+------------------------+------------------+------------------+
|[Organic Kiwi, Organic Hass Avocado]                             |[Bag of Organic Bananas]|0.5459770114942529|4.627719489738336 |
|[Organic Raspberries, Organic Hass Avocado, Organic Strawberries]|[Bag of Organic Bananas]|0.5984251968503937|5.072272070642333 |
|[Organic Broccoli, Organic Hass Avocado]                         |[Bag of Organic Bananas]|0.5048231511254019|4.278897986822536 |
|[Organic Unsweetened Almond Milk, Organic Hass Avocado]          |[Bag of Organic Bananas]|0.5141065830721003|4.357584667849303 |
|[Yellow Onions, Strawberries]                                    |[Banana]        

5. What can be inferred from the association rules? Write a comment you can get from the rules.

In [ ]:
# YOUR TEXT HERE
# (IN ENGLISH OR KOREAN)
# 대부분의 발견된 연관 법칙이 Organic 상품을 사면 다른 Bag of Organic Banana 상품을 구매하는 경우이다.
# 따라서, Bag of Organic Banana를 중심으로 Organic 상품들은 Organic코너를 따로 만들어 배치한다면
# 연관성이 높은 추가상품 구매를 유도하기에 적합하다고 생각한다.